[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dina-lab3D/tutorials/blob/main/PatchDock/docking.ipynb)

#Antigen-nanobody docking

<img src="https://drive.google.com/uc?export=view&id=1J6FA12dZKcf7ydSQdnFo8b7bwi-NehB5" width="300">
<img src="https://drive.google.com/uc?export=view&id=1J83QaZWoK1Xi2-dqt7o8jdImcJ8HDJf-" width="150">

In this tutorial we use a docking software to dock a modeled nanobody to an antigen. We will work in 3? steps:


1. Docking using PatchDock with predicted epitopes (ScanNet) and CDRs of a nanobody
2. Scoring using interface scoring function (SOAP/ContactNet) 



In [ ]:
#@title Download docking software - PatchDock
!wget https://github.com/dina-lab3D/tutorials/raw/main/PatchDock/pd.zip
!unzip pd.zip

In [ ]:
#@title Prepare input for docking
!wget https://github.com/dina-lab3D/tutorials/raw/main/PatchDock/dd.zip
! unzip dd.zip

In [ ]:
ab_pdb=input("Enter antibody pdb")
ag_pdb=input("Enter antigen pdb")

In [ ]:
% cd /content/NbSAXSexample/
!ls
# dock the antigen (ag_pdb) to the antibody model (ab_pdb), final clustering of 4A, "AA" tells PatchDock to use antigen-antibody docking and focus on nanobody CDRs 
! ../PatchDock/buildParams.pl ag_pdb ab_pdb 4.0 AA
# IMPORTANT: now edit the params.txt file to include ScanNet epitopes.txt (uncomment receptorActiveSite)
!echo "receptorActiveSite epitopes.txt" >> params.txt

In [ ]:
#@title run PatchDock
! ../PatchDock/patch_dock.Linux params.txt docking.res
# generate top 10 by geometric score
! ../PatchDock/transOutput.pl docking.res 1 10

In [ ]:
! pwd
!pip install py3Dmol
#@title Display 3D structure {run: "auto"}
import py3Dmol
import glob
import matplotlib.pyplot as plt

with open("docking.res.1.pdb") as ifile:
    system = "".join([x for x in ifile])

view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(system)
view.setStyle({'model': -1}, {"cartoon": {'color': 'spectrum'}})
view.zoomTo()
view.show()



In [ ]:
#@title install ContactNet
% cd /content/
!git clone https://github.com/dina-lab3D/ContactNet --quiet
!pip install tensorflow_addons
% cd /content/NbSAXSexample/
!pwd

In [ ]:
#@title run ContactNet scoring
!python3 /content/ContactNet/data_maker.py ag_pdb ab_pdb docking.res --trans_num 1411
!python3 /content/ContactNet/predict_prot.py ag_pdb ab_pdb --trans_num 1411

In [ ]:
#@title  SOAP - statistical potential
# prepare transformation file
! grep "|" docking.res | cut -d '|' -f1,17 --output-delimiter=' '| grep -v Trans > trans
#! head trans
# run SOAP
! soap_score ag_pdb ab_pdb trans

In [ ]:
# create a file with all the scores
!grep "|" docking.res | awk '{print $1" "$3}' > rmsds
!grep "|" soap_score.res | awk '{print $3}' > soap
!cut -f2 evaluation > cnet
!paste -d ' ' rmsds cnet soap  > all_scores.txt
!sort -t ' ' -nrk5 all_scores.txt | head -679 | sort -nk1 > all_filtered.txt


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from plotnine import *
from scipy import stats
from scipy.interpolate import make_interp_spline

df = pd.read_csv("all_scores.txt", sep=" ")
print (df)
sns.set_theme(style="whitegrid")
sns.set(font_scale = 2)
sns.set_style("whitegrid")



fig1 = plt.figure(1, figsize=(12, 4), dpi=100)


axes1 = fig1.add_subplot(131)
axes1 = sns.scatterplot(data = df,x="rmsd", y="score")
axes1.set(title= 'ContactNet')


axes2 = fig1.add_subplot(132)
axes2 = sns.scatterplot(data = df,x="rmsd", y="SOAP")
axes2.set(title= 'SOAP')

plt.show()
plt.savefig('scatter.png')

# filter all models according to SAXS (chi = 0)
df = df[df["Chi"] > 0].reset_index(level=None)

# reverse ContactNet scores (now lower is better)
df["score"] = -df["score"]

# normalize all to [0-1] range
for score_name in ["score", "SOAP", "Chi"]:
  df[score_name] = (df[score_name] - np.min( df[score_name])) / (np.max(df[score_name]) - np.min(df[score_name]))

# sum up the three scores
df["sum"] = df["Chi"] + df["SOAP"] + df["score"]

# select best model
best_model = int(df["#"].iloc[df["sum"].argmin()])
print(df.iloc[df["sum"].argmin()])
print(best_model)

fig1 = plt.figure(1, figsize=(4, 4), dpi=100)
axes4 = fig1.add_subplot(111)
axes4 = sns.scatterplot(data = df,x="rmsd", y="sum")
axes4.set(title= 'Combined score')
plt.show()




In [ ]:
#@title Display best scoring
! /content/PatchDock/transOutput.pl docking.res $best_model $best_model
with open(ab_pdb) as ifile:
    true = "".join([x for x in ifile])
with open(f"docking.res.{best_model}.pdb") as ifile:
    predicted = "".join([x for x in ifile])


r,g,b = 255,0,0
print(f"\033[38;2;{r};{g};{b}mPredicted model \033[38;2;255;255;255m")
r,g,b = 0,0,255
print(f"\033[38;2;{r};{g};{b}mNative structure \033[38;2;255;255;255m")
view = py3Dmol.view(width=500, height=500)
view.addModelsAsFrames(predicted)
view.setStyle({'model': 0, "chain": "A"}, {"cartoon": {'arrows':True, 'color': 'gray'}})
view.setStyle({'model': 0, "chain": "H"}, {"cartoon": {'arrows':True, 'color': 'red'}})
view.addModelsAsFrames(true)
view.setStyle({'model': 1}, {"cartoon": {'arrows':True, 'color': 'blue'}})
view.zoomTo()
view.show()
